In [3]:
import torch
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
from gc_celeba_gan import *

from torchvision.datasets import CelebA
from torchvision import transforms, models
from torchmetrics.image.fid import FrechetInceptionDistance

print(torch.cuda.is_available())
torch.zeros(1).cuda()
torch.set_float32_matmul_precision('high')

True


In [4]:
paths = [f'out_CelebAGAN_20_nan_nan_0.0100_0.0000_0.0000/version_{i}/checkpoints/epoch=19-step=76320.ckpt' for i in range(5)]

fid = FrechetInceptionDistance(feature=64, reset_real_features=False, normalize=True)

transformlist = transforms.Compose([
    transforms.Resize(size=64),
    transforms.CenterCrop(size=(64, 64)),
    transforms.ToTensor(),
])
dataset = CelebA(os.getcwd(), download=True, target_type='attr', transform=transformlist)
dataloader = DataLoader(dataset, batch_size=128, num_workers=48, persistent_workers=True)

print(len(dataset))

print('loading real data into FID')
n = 0
for img, _ in dataloader:
    print(n)
    fid.update(img, real=True)
    n += 128
    if n == 128:
        break
print('done')

for path in paths:
    model = CelebAGAN(float('nan'), float('nan'), 0.0, 0.0, 0)
    checkpoint = torch.load(path)
    #del checkpoint["state_dict"]["logcontvar"]
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    print('model ready!')

    fid.reset()
    n = 0
    while n < 128:
        print(n)
        imgs = model.sample_G(128).detach()
        fid.update(imgs, real=False)
        n += 128
    print(fid.compute())

Files already downloaded and verified
162770
loading real data into FID
0
done
Files already downloaded and verified
model ready!
0
tensor(1.4160)
Files already downloaded and verified
model ready!
0
tensor(0.8456)
Files already downloaded and verified
model ready!
0
tensor(0.7179)
Files already downloaded and verified
model ready!
0
tensor(1.6320)
Files already downloaded and verified
model ready!
0
tensor(0.7010)
